In [1]:
!pip install rasterio

!pip install geopandas
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from itertools import product
import logging

# Thiết lập logging
logging.basicConfig(filename=os.path.join("/kaggle/working/retrain_results_all_params", 'errors.log'), level=logging.ERROR)

# Định nghĩa các thông số
OUTPUT_PATH = "/kaggle/working/"
os.makedirs(OUTPUT_PATH, exist_ok=True)
CSV_PATH = "/kaggle/input/data-full-features-ai/weather_data_nghean (1).csv"

# Các đặc trưng đã chọn
SELECTED_HIMA_BANDS = ['B04B', 'B05B', 'B06B', 'B09B', 'B10B', 'B11B', 'B12B', 'B14B', 'B16B', 'I2B', 'I4B', 'IRB', 'VSB', 'WVB']
SELECTED_ERA5_PARAMS = ['CAPE', 'CIN', 'EWSS', 'IE', 'ISOR', 'KX', 'PEV', 'R250', 'R500', 'R850', 'SLHF', 'SLOR', 'SSHF', 'TCLW', 'TCW', 'TCWV', 'U250', 'U850', 'V250', 'V850']
SELECTED_FEATURES = SELECTED_HIMA_BANDS + SELECTED_ERA5_PARAMS
HEIGHT, WIDTH = 90, 250

# Đọc dữ liệu từ CSV
print("Đang đọc dữ liệu từ CSV...")
df = pd.read_csv(CSV_PATH)
if df.empty:
    raise ValueError("File CSV rỗng hoặc không tồn tại!")

# Chuẩn bị dữ liệu
X = df[SELECTED_FEATURES]
y = df['Radar']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Định nghĩa lưới tham số (rút gọn để tiết kiệm thời gian)
rf_param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10],
    'max_samples': [0.5, 1.0],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 0.5]
}

xgb_param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 6],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.6, 1.0],
    'colsample_bytree': [0.6, 1.0],
    'min_child_weight': [1, 3],
    'gamma': [0, 0.1]
}

# Lưu kết quả đánh giá
evaluation_results = []

# Hàm vẽ biểu đồ và lưu kết quả
def plot_and_save_results(model, model_name, params, y_train_pred, y_test_pred, X_train, X_test, y_train, y_test, df, output_dir, train_losses=None, val_losses=None):
    try:
        os.makedirs(output_dir, exist_ok=True)

        # Đánh giá
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
        test_r2 = r2_score(y_test, y_test_pred)
        
        print(f"\nĐánh giá {model_name} với tham số {params}:")
        print(f"Train RMSE: {train_rmse:.4f}")
        print(f"Test RMSE: {test_rmse:.4f}")
        print(f"R2 Score (Test): {test_r2:.4f}")

        # Lưu kết quả đánh giá
        evaluation_results.append({
            "model": model_name,
            "params": str(params),
            "train_rmse": train_rmse,
            "test_rmse": test_rmse,
            "test_r2": test_r2
        })

        # Vẽ đường cong tổn thất (chỉ cho XGBoost)
        if train_losses and val_losses:
            plt.figure(figsize=(10, 6))
            plt.plot(train_losses, label='Train Loss')
            plt.plot(val_losses, label='Val Loss')
            plt.xlabel('Iteration')
            plt.ylabel('RMSE Loss')
            plt.title(f'{model_name} Loss Curve')
            plt.legend()
            plt.savefig(os.path.join(output_dir, 'loss_curve.png'))
            plt.close()

        # Vẽ bản đồ lượng mưa
        sample_dt = df['datetime'].sample(1).iloc[0]
        sample_df = df[df['datetime'] == sample_dt].copy()
        sample_df['prediction'] = np.nan
        test_indices = X_test.index
        for idx in test_indices:
            if idx in sample_df.index:
                sample_df.loc[idx, 'prediction'] = y_test_pred[list(test_indices).index(idx)]

        ground_truth_map = np.zeros((HEIGHT, WIDTH))
        prediction_map = np.zeros((HEIGHT, WIDTH))
        for _, row in sample_df.iterrows():
            row_idx = int((row['y'] - df['y'].min()) / (df['y'].max() - df['y'].min()) * HEIGHT)
            col_idx = int((row['x'] - df['x'].min()) / (df['x'].max() - df['x'].min()) * WIDTH)
            if 0 <= row_idx < HEIGHT and 0 <= col_idx < WIDTH:
                ground_truth_map[row_idx, col_idx] = row['Radar']
                if not np.isnan(row['prediction']):
                    prediction_map[row_idx, col_idx] = row['prediction']

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), subplot_kw={'projection': ccrs.PlateCarree()})
        ax1.set_title(f'Ground Truth - {sample_dt}')
        ax2.set_title(f'Prediction ({model_name})')
        for ax, data in [(ax1, ground_truth_map), (ax2, prediction_map)]:
            ax.coastlines()
            ax.add_feature(cfeature.BORDERS)
            im = ax.imshow(data, cmap='Blues', origin='upper', transform=ccrs.PlateCarree(),
                           extent=(df['x'].min(), df['x'].max(), df['y'].min(), df['y'].max()))
            plt.colorbar(im, ax=ax, label='Lượng mưa (mm/h)')
        plt.savefig(os.path.join(output_dir, f'rainfall_map_{sample_dt.replace(":", "-")}.png'))
        plt.close()

        # Vẽ biểu đồ phân tán
        plt.figure(figsize=(8, 8))
        plt.scatter(y_test, y_test_pred, alpha=0.5, label='Dữ liệu')
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Đường y=x')
        plt.xlabel('Ground Truth (Lượng mưa mm/h)')
        plt.ylabel('Dự đoán (Lượng mưa mm/h)')
        plt.title(f'So sánh Ground Truth và Dự đoán {model_name}')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(output_dir, 'ground_truth_vs_prediction_scatter.png'))
        plt.close()

        # Vẽ biểu đồ tầm quan trọng của đặc trưng
        importance = model.feature_importances_
        feature_importance = pd.DataFrame({
            'feature': SELECTED_FEATURES,
            'importance': importance
        }).sort_values(by='importance', ascending=False)

        plt.figure(figsize=(10, 8))
        sns.barplot(x='importance', y='feature', data=feature_importance)
        plt.title(f'Feature Importance từ {model_name}')
        plt.xlabel('Importance')
        plt.ylabel('Feature')
        plt.savefig(os.path.join(output_dir, 'feature_importance.png'))
        plt.close()

    except Exception as e:
        logging.error(f"Lỗi khi lưu kết quả cho {model_name} với tham số {params}: {str(e)}")
        print(f"Lỗi khi lưu kết quả cho {model_name}: {str(e)}")

# Tạo danh sách tổ hợp tham số cho Random Forest
rf_combinations = list(product(
    rf_param_grid['n_estimators'],
    rf_param_grid['max_depth'],
    rf_param_grid['max_samples'],
    rf_param_grid['min_samples_split'],
    rf_param_grid['min_samples_leaf'],
    rf_param_grid['max_features']
))

# Huấn luyện và đánh giá Random Forest
print(f"Tổng số tổ hợp Random Forest: {len(rf_combinations)}")
for i, (n_estimators, max_depth, max_samples, min_samples_split, min_samples_leaf, max_features) in enumerate(rf_combinations):
    params = {
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'max_samples': max_samples,
        'min_samples_split': min_samples_split,
        'min_samples_leaf': min_samples_leaf,
        'max_features': max_features
    }
    print(f"Huấn luyện Random Forest tổ hợp {i+1}/{len(rf_combinations)}")
    try:
        rf_model = RandomForestRegressor(**params, random_state=42)
        rf_model.fit(X_train, y_train)
        y_train_pred = rf_model.predict(X_train)
        y_test_pred = rf_model.predict(X_test)
        output_dir = os.path.join(OUTPUT_PATH, f"rf_comb_{i+1}")
        plot_and_save_results(rf_model, "RandomForest", params, y_train_pred, y_test_pred, X_train, X_test, y_train, y_test, df, output_dir)
    except Exception as e:
        logging.error(f"Lỗi khi huấn luyện Random Forest với tham số {params}: {str(e)}")
        print(f"Lỗi khi huấn luyện Random Forest tổ hợp {i+1}: {str(e)}")

# Tạo danh sách tổ hợp tham số cho XGBoost
xgb_combinations = list(product(
    xgb_param_grid['n_estimators'],
    xgb_param_grid['max_depth'],
    xgb_param_grid['learning_rate'],
    xgb_param_grid['subsample'],
    xgb_param_grid['colsample_bytree'],
    xgb_param_grid['min_child_weight'],
    xgb_param_grid['gamma']
))

# Huấn luyện và đánh giá XGBoost
print(f"Tổng số tổ hợp XGBoost: {len(xgb_combinations)}")
for i, (n_estimators, max_depth, learning_rate, subsample, colsample_bytree, min_child_weight, gamma) in enumerate(xgb_combinations):
    params = {
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'learning_rate': learning_rate,
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'min_child_weight': min_child_weight,
        'gamma': gamma
    }
    print(f"Huấn luyện XGBoost tổ hợp {i+1}/{len(xgb_combinations)}")
    try:
        xgb_model = XGBRegressor(**params, random_state=42)
        eval_set = [(X_train, y_train), (X_test, y_test)]
        xgb_model.fit(X_train, y_train, eval_metric="rmse", eval_set=eval_set, verbose=False)
        y_train_pred = xgb_model.predict(X_train)
        y_test_pred = xgb_model.predict(X_test)
        results = xgb_model.evals_result()
        train_losses = results['validation_0']['rmse']
        val_losses = results['validation_1']['rmse']
        output_dir = os.path.join(OUTPUT_PATH, f"xgb_comb_{i+1}")
        plot_and_save_results(xgb_model, "XGBoost", params, y_train_pred, y_test_pred, X_train, X_test, y_train, y_test, df, output_dir, train_losses, val_losses)
    except Exception as e:
        logging.error(f"Lỗi khi huấn luyện XGBoost với tham số {params}: {str(e)}")
        print(f"Lỗi khi huấn luyện XGBoost tổ hợp {i+1}: {str(e)}")

# Vẽ ma trận tương quan
correlation_matrix = df[SELECTED_FEATURES + ['Radar']].corr()
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0)
plt.title('Ma trận tương quan giữa các đặc trưng và lượng mưa')
plt.savefig(os.path.join(OUTPUT_PATH, 'correlation_matrix.png'))
plt.close()

# In tương quan với lượng mưa
precip_correlations = correlation_matrix['Radar'].drop('Radar')
print("\nTương quan với lượng mưa (Radar):")
print(precip_correlations.sort_values(ascending=False))

# Lưu kết quả đánh giá vào CSV
eval_df = pd.DataFrame(evaluation_results)
eval_df.to_csv(os.path.join(OUTPUT_PATH, 'evaluation_results.csv'), index=False)
print("\nĐã lưu kết quả đánh giá vào evaluation_results.csv")

# Tìm tổ hợp tham số tốt nhất
if evaluation_results:
    best_result = min(evaluation_results, key=lambda x: x['test_rmse'])
    print("\nTổ hợp tham số tốt nhất (dựa trên test_rmse thấp nhất):")
    print(f"Model: {best_result['model']}")
    print(f"Params: {best_result['params']}")
    print(f"Test RMSE: {best_result['test_rmse']:.4f}")
    print(f"Test R2: {best_result['test_r2']:.4f}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 75.6 MB/s eta 0:00:00
Đang đọc dữ liệu từ CSV...
Tổng số tổ hợp Random Forest: 64
Huấn luyện Random Forest tổ hợp 1/64

Đánh giá RandomForest với tham số {'n_estimators': 50, 'max_depth': 5, 'max_samples': 0.5, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt'}:
Train RMSE: 1.0285
Test RMSE: 1.0300
R2 Score (Test): 0.3933


/usr/local/lib/python3.11/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_physical/ne_10m_coastline.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/usr/local/lib/python3.11/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_cultural/ne_10m_admin_0_boundary_lines_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)


Huấn luyện Random Forest tổ hợp 2/64

Đánh giá RandomForest với tham số {'n_estimators': 50, 'max_depth': 5, 'max_samples': 0.5, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 0.5}:
Train RMSE: 0.9886
Test RMSE: 0.9878
R2 Score (Test): 0.4420
Huấn luyện Random Forest tổ hợp 3/64

Đánh giá RandomForest với tham số {'n_estimators': 50, 'max_depth': 5, 'max_samples': 0.5, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt'}:
Train RMSE: 1.0304
Test RMSE: 1.0289
R2 Score (Test): 0.3946
Huấn luyện Random Forest tổ hợp 4/64

Đánh giá RandomForest với tham số {'n_estimators': 50, 'max_depth': 5, 'max_samples': 0.5, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 0.5}:
Train RMSE: 0.9887
Test RMSE: 0.9887
R2 Score (Test): 0.4409
Huấn luyện Random Forest tổ hợp 5/64

Đánh giá RandomForest với tham số {'n_estimators': 50, 'max_depth': 5, 'max_samples': 0.5, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt'}:
Train RMSE: 1.0270


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(



Đánh giá XGBoost với tham số {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.01, 'subsample': 0.6, 'colsample_bytree': 0.6, 'min_child_weight': 1, 'gamma': 0}:
Train RMSE: 1.2576
Test RMSE: 1.2193
R2 Score (Test): 0.1497
Huấn luyện XGBoost tổ hợp 2/128


...


Đánh giá XGBoost với tham số {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.1, 'subsample': 1.0, 'colsample_bytree': 1.0, 'min_child_weight': 3, 'gamma': 0}:
Train RMSE: 0.6455
Test RMSE: 0.7401
R2 Score (Test): 0.6867
Huấn luyện XGBoost tổ hợp 128/128


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(



Đánh giá XGBoost với tham số {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.1, 'subsample': 1.0, 'colsample_bytree': 1.0, 'min_child_weight': 3, 'gamma': 0.1}:
Train RMSE: 0.6405
Test RMSE: 0.7436
R2 Score (Test): 0.6838


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1



Tương quan với lượng mưa (Radar):
TCLW    0.213307
TCW     0.193137
TCWV    0.183497
R500    0.173187
VSB     0.167600
B04B    0.145432
R850    0.105617
B06B    0.093121
PEV     0.091359
SSHF    0.091236
KX      0.086522
SLHF    0.065484
R250    0.056746
IE      0.055531
B05B    0.006976
CIN     0.000258
ISOR   -0.011342
V250   -0.025946
CAPE   -0.045808
EWSS   -0.053219
SLOR   -0.068709
U250   -0.127304
V850   -0.135154
WVB    -0.175059
U850   -0.193968
B09B   -0.195891
B10B   -0.204308
B12B   -0.218475
I2B    -0.218776
I4B    -0.218969
B16B   -0.221843
B14B   -0.224419
B11B   -0.227238
IRB    -0.227383
Name: Radar, dtype: float64

Đã lưu kết quả đánh giá vào evaluation_results.csv

Tổ hợp tham số tốt nhất (dựa trên test_rmse thấp nhất):
Model: XGBoost
Params: {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.1, 'subsample': 1.0, 'colsample_bytree': 1.0, 'min_child_weight': 3, 'gamma': 0}
Test RMSE: 0.7401
Test R2: 0.6867
